In [1]:
import pandas as pd
from data import Urg2022_pub, UrgP2022, Urg2022_priv_luc, Urg2022_priv_non_luc

In [ ]:
###---Productivité des médecins---

#Nombre de passages aux urgences par médecin en 2022 selon le secteur 

#Extraction du nombre d'ETP de médecins
Medecins2022 = UrgP2022[UrgP2022["PERSO"].isin(["M1000","M2000","M3020"])]
ETP_Medecins2022 = Medecins2022.groupby("FI",as_index=False)["ETPSAL"].sum()


<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 31 to 4144
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BOR      51 non-null     object 
 1   AN       51 non-null     int64  
 2   FI       51 non-null     object 
 3   RS       51 non-null     object 
 4   FI_EJ    51 non-null     object 
 5   PERSO    51 non-null     object 
 6   ETPSAL   50 non-null     float64
 7   EFFLIB   12 non-null     float64
 8   MOD      0 non-null      float64
 9   COMSMUR  0 non-null      float64
dtypes: float64(4), int64(1), object(5)
memory usage: 4.4+ KB


In [ ]:

#Rassemble les médecins ETP avec le nombre de passages aux urgences selon le type d'établissement
#et calcule le ratio passages aux urgences/ETP médecins
Passages_pub = Urg2022_pub[["FI","PASSU"]]
Passages_pub = Passages_pub.groupby("FI",as_index=False)["PASSU"].sum()
Passages_pub = Passages_pub.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_pub["ratio"] = Passages_pub["PASSU"]/Passages_pub["ETPSAL"]
Passages_pub["ratio"].describe()



,FI,PASSU,ETPSAL
0,010000024,42384,NaN
1,010000032,16727,NaN
2,010005239,22583,NaN
3,020000162,56178,1.0
4,020000394,34479,NaN


In [31]:
#pour le privé prendre EFFLIB et non PERSO car sont libéraux...
Passages_priv_luc = Urg2022_priv_luc[["FI","PASSU"]]
Passages_priv_luc = Passages_priv_luc.merge(UrgP2022, left_on="FI", right_on="FI", how="left")
Passages_priv_luc = Passages_priv_luc.groupby('FI').agg(
    somme_sal=('ETPSAL', 'sum'), 
    somme_lib=('EFFLIB', 'sum'),
    somme_urg=('PASSU','sum')
)
Passages_priv_luc["ratio"] = Passages_priv_luc["somme_urg"]/(Passages_priv_luc["somme_sal"]+Passages_priv_luc["somme_lib"])
Passages_priv_luc["ratio"].describe()

count       121.000000
mean       4057.333572
std       12429.938308
min         469.011725
25%        2225.047081
50%        2904.208494
75%        3511.843575
max      139212.500000
Name: ratio, dtype: float64

In [ ]:

Passages_priv_luc["ratio"] = Passages_priv_luc["PASSU"]/Passages_priv_luc["EFFLIB"]
Passages_priv_luc["ratio"].describe()


In [ ]:
Passages_priv_non_luc = Urg2022_priv_non_luc[["FI","PASSU"]]
Passages_priv_non_luc = Passages_priv_non_luc.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
ratio_priv_non_luc = Passages_priv_non_luc["PASSU"]/Passages_priv_non_luc["ETPSAL"]
ratio_priv_non_luc.describe()



count         2.000000
mean     355291.666667
std      325860.732019
min      124873.333333
25%      240082.500000
50%      355291.666667
75%      470500.833333
max      585710.000000
dtype: float64